In [64]:
# 셋팅 하기 - 필요한 모듈/라이브러리 모두 호출하기!
import os
import sys
import urllib.request
import datetime
import time
import json

In [65]:
# 네이버 개발자에서 생성한 API 호출 ID, PWD 가져오기
client_id = "eqTP5XcwlDYyaCiKB7my"
client_secret = "HU1HBaixz7"

In [81]:
# [code 1]

# getRequestUrl 함수를 정의하여서 생성
def getRequestUrl(url):
    
    # urlib.request.Request(url) => Url 요청 = 호출
    req = urllib.request.Request(url)  # url 불러오기
    req.add_header("X-Naver-Client-id", client_id)  # Naver api 사용 .id를 검색 url header에 넣어줘야 함
    req.add_header("X-Naver-Client-Secret", client_secret)
    
    # 예외처리문으로 구성해서 네이버 서버의 응답을 확인
    # 서버 코드 200 = Success!
    
    try: # 실행
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:  # 성공적 응답은 서버 코드 200
            print("[%s] Url Request Success" %datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e: # 에러 예외처리
        print(e)
        print("[%s] Error for URL : % s" %(datetime.datetime.now(), url))
        return None

In [82]:
# [code 2]

def getNaverSearch(node, scrText, start, display): # node = 뉴스....
    # 네이버 검색 API 정보로 URL을 구성한다.
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" %node
    parameters = "?query=%s&start=%s&display=%s" %(urllib.parse.quote(scrText), start, display)
    
    url = base + node + parameters
    
    # 만들어진 URL을 이용하여 getRequestUrl() 호출해서 받은 utf-8 디코드 응답을 responseDecode에 저장
    responseDecode = getRequestUrl(url) # [code 1]
    
    if (responseDecode ==  None):
        return None
    else:
        return json.loads(responseDecode) 
    # 서버에서 받은 Json 형태의 응답객체를 파이썬 객체로 로드하여 반환

In [83]:
# [code 3]

# getPostData함수를 정의해서 검색결과에서 필요한 데이터 형식의 
def getPostData(post, jsonResult, cnt): #cnt = 검색된 count 숫자
    title = post['title']
    description = post['description']
    org_link = post['originallink']
    link = post['link']
    
    # 네이버에서 제공하는 시간형식을 우리가 필요한 형식으로 전환
    # Tue, 14 Feb 2017 18:46:00 = 0900
    pDate = datetime.datetime.strptime(post['pubDate'], '%a, %d %b %Y %H:%M:%S +0900')
    pDate = pDate.strftime('%Y-%m-%d %H:%M:%S') # 연-월-일, 시:분:초
    
    jsonResult.append({'cnt':cnt, 'title':title, 'description': description, 'org_link':org_link, 'link':link, 'pDate':pDate})
    # 딕셔너리 형태의 {키:값}으로 구성해서 jsonResult로 구성
    return
                                       

In [88]:
def main():
    node = "news" #크롤링할 대상
    srcText = input('검색어를 입력하세요: ')
    cnt = 0
    jsonResult = []
    
    jsonResponse = getNaverSearch(node, srcText, 1, 100) # [Code 2]
    total = jsonResponse['total']
    
    while((jsonResponse !=None) and (jsonResponse['display'] != 0)):
        for post in jsonResponse['items']:
            cnt +=1
            getPostData(post, jsonResult, cnt) #[Code 3]
            
        start = jsonResponse['start'] + jsonResponse['display']
        jsonResponse = getNaverSearch(node, srcText, start, 100) #[Code 2]
        
    print('전체 검색: %d 건' %total)
    
    with open('%s_naver_%s.json' % (srcText, node), 'w', encoding='utf-8') as outfile:
        jsonFile = json.dumps(jsonResult, indent=4, sort_keys= True, ensure_ascii=False)
        outfile.write(jsonFile) # json 파일로 저장하기
        
    print("가져온 데이터: %d건" %(cnt))
    print('%s_naver_%s.json SAVED' % (srcText, node))

In [90]:
main()

검색어를 입력하세요: 오징어게임
[2021-10-26 11:54:14.359992] Url Request Success
[2021-10-26 11:54:14.469030] Url Request Success
[2021-10-26 11:54:14.601395] Url Request Success
[2021-10-26 11:54:14.718168] Url Request Success
[2021-10-26 11:54:14.852562] Url Request Success
[2021-10-26 11:54:14.949960] Url Request Success
[2021-10-26 11:54:15.088420] Url Request Success
[2021-10-26 11:54:15.204412] Url Request Success
[2021-10-26 11:54:15.321528] Url Request Success
[2021-10-26 11:54:15.429528] Url Request Success
HTTP Error 400: Bad Request
[2021-10-26 11:54:15.568401] Error for URL : https://openapi.naver.com/v1/search/news.json?query=%EC%98%A4%EC%A7%95%EC%96%B4%EA%B2%8C%EC%9E%84&start=1001&display=100
전체 검색: 24476 건
가져온 데이터: 1000건
오징어게임_naver_news.json SAVED
